<a href="https://colab.research.google.com/github/sarika9377/Training/blob/master/Capstone1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab's New Code Editor

Colab is moving to a new code editor which includes:
* Richer completions
* Additional keybinding options
* Improved accessibility

# Richer syntax highlighting

Improved support for nested languages:

```notebook-python
df = pd.io.gbq.read_gbq('''
  SELECT 
    REGEXP_EXTRACT(name, '[a-zA-Z]+'),
    SUM(number) as count
  FROM `bigquery-public-data.usa_names.usa_1910_2013`
  WHERE state = 'TX'
  GROUP BY name
  ORDER BY count DESC
  LIMIT 100
''')
```

In [0]:
import mysql.connector as mysql #if it gives error, use: pip install mysql.connector
import csv
import pandas as pd
import numpy as np

#Creating the connection string to connect to INSAID's mysql db

dbconnectionstring = mysql.connect(
                      host = 'cpanel.insaid.co',
                      user = 'student',
                      passwd = 'student',
                      database = 'Capstone1' )

cursor = dbconnectionstring.cursor()

In [5]:
cursor.execute("SHOW TABLES")
tables=cursor.fetchall()
for table_name in tables:
    print(table_name)

('events_data',)
('gender_age_train',)
('phone_brand_device_model',)


In [6]:
query = "show columns from events_data"
cursor.execute(query)
result=cursor.fetchall()
dbconnectionstring.close()
result

[('event_id', 'bigint(20)', 'NO', '', None, ''),
 ('device_id', 'bigint(255)', 'YES', '', None, ''),
 ('timestamp', 'datetime', 'NO', '', None, ''),
 ('longitude', 'float(8,6)', 'YES', '', None, ''),
 ('latitude', 'float(8,6)', 'YES', '', None, ''),
 ('city', 'varchar(50)', 'NO', '', None, ''),
 ('state', 'varchar(50)', 'NO', '', None, '')]

In [8]:
# Table 2 - gender_age_train - Get Column names

dbconnectionstring = mysql.connect(
                      host = 'cpanel.insaid.co',
                      user = 'student',
                      passwd = 'student',
                      database = 'Capstone1' )

cursor = dbconnectionstring.cursor()

query = "show columns from gender_age_train"
cursor.execute(query)
result=cursor.fetchall()
dbconnectionstring.close()
result

[('device_id', 'bigint(255)', 'NO', '', None, ''),
 ('gender', 'varchar(2555)', 'NO', '', None, ''),
 ('age', 'int(255)', 'NO', '', None, ''),
 ('group', 'varchar(2555)', 'NO', '', None, '')]

In [9]:
# Table 3 - phone_brand_device_model - Get Column names

dbconnectionstring = mysql.connect(
                      host = 'cpanel.insaid.co',
                      user = 'student',
                      passwd = 'student',
                      database = 'Capstone1' )

cursor = dbconnectionstring.cursor()

query = "show columns from phone_brand_device_model"
cursor.execute(query)
result=cursor.fetchall()
dbconnectionstring.close()
result

[('device_id', 'bigint(255)', 'NO', '', None, ''),
 ('phone_brand', 'varchar(2555)', 'NO', '', None, ''),
 ('device_model', 'varchar(2555)', 'NO', '', None, '')]

In [10]:
dbconnectionstring = mysql.connect(
                      host = 'cpanel.insaid.co',
                      user = 'student',
                      passwd = 'student',
                      database = 'Capstone1' )

cursor = dbconnectionstring.cursor()

query = "select count(*) from phone_brand_device_model"
cursor.execute(query)
pcount = cursor.fetchall()
dbconnectionstring.close()
pcount


[(87726,)]

In [11]:
dbconnectionstring = mysql.connect(
                      host = 'cpanel.insaid.co',
                      user = 'student',
                      passwd = 'student',
                      database = 'Capstone1' )

cursor = dbconnectionstring.cursor()

query = "select count(*) from gender_age_train"
cursor.execute(query)
pcount = cursor.fetchall()
dbconnectionstring.close()
pcount


[(74645,)]

In [0]:
# Get all records for the concerned States from events_data

if dbconnectionstring.is_connected():
    dbconnectionstring.close()
    
dbconnectionstring = mysql.connect(
                      host = 'cpanel.insaid.co',
                      user = 'student',
                      passwd = 'student',
                      database = 'Capstone1' )

cursor = dbconnectionstring.cursor()

query = '''select *
             from events_data as ed
            where state in ('MadhyaPradesh', 'Goa', 'Chhattisgarh','Uttaranchal', 'JammuandKashmir', 'Nagaland')'''
cursor.execute(query)
events_data=cursor.fetchall()
ds_events_data= pd.DataFrame(events_data)
ds_events_data.columns=[i[0] for i in cursor.description]
dbconnectionstring.close()

In [13]:
ds_events_data.head()

,event_id,device_id,timestamp,longitude,latitude,city,state
0,10117,-5.323073e+18,2016-05-01 11:11:24,75.939545,22.769171,Indore,MadhyaPradesh
1,112369,-5.323073e+18,2016-05-04 22:47:19,75.939545,22.769171,Indore,MadhyaPradesh
2,139098,-5.323073e+18,2016-05-01 09:55:58,75.939545,22.769171,Indore,MadhyaPradesh
3,190983,-5.323073e+18,2016-05-04 22:51:03,75.939545,22.769171,Indore,MadhyaPradesh
4,249507,-5.323073e+18,2016-05-01 11:24:12,75.939545,22.769171,Indore,MadhyaPradesh


In [14]:
# Create a unique list of device ids from event data, to download only those records from the other 2 tables.

device_ids = ds_events_data['device_id'].unique()    # Extracts unique device ids
dids=device_ids.tolist()                           # Converts nparray to list
list_of_ids = [x for x in dids if str(x) != 'nan'] # Drop all 'nan' ids
len(list_of_ids)                                   # list the number of ids.

4297